# Notebook to run the gaussian regularized alghorithms 

In [ ]:
from prob_models.utils.utils import read_paths, getdata

# read_add_paths('paths.txt')
# paths = read_paths('paths.txt')

import prob_models.cov_estim.estimators as es 
import prob_models.cov_estim.run_parallel as run
from prob_models.cov_estim.utils import *
import numpy as np
import os
import importlib

# importlib.reload(ourutils)
# importlib.reload(run)

In [2]:
nsamples = 200
nfeatures = 116

X = np.random.randn(nsamples, nfeatures)  # dummy dataset with true covariance I
Xtrain = X[:int(0.8*nsamples)]
Xtest = X[int(0.8*nsamples):]

res = es.fit_PCA(Xtrain, Xtest, ncomponents=10)
# res = es.fit_PCA_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_PCA_Minka(Xtrain, Xtest)
# res = es.fit_ConservativePCA(Xtrain, Xtest, p=10)
# res = es.fit_ConservativePCA_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_FactorAnalysis(Xtrain, Xtest)
# res = es.fit_FactorAnalysis_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_GraphicalLasso(Xtrain, Xtest, alpha=0.1)
# res = es.fit_GraphicalLasso_CV(Xtrain, Xtest, n_jobs=None)
# res = es.fit_Shrinkage(Xtrain, Xtest, shrinkage=0.1)
# res = es.fit_Shrinkage_CV(Xtrain, Xtest, shrinkages=None,  cv_scoring="likelihood", n_jobs=None)
# res = es.fit_RIE(Xtrain, Xtest)
# res = es.fit_RIE_CV(Xtrain, Xtest, cv_scoring="likelihood", n_jobs=None)
# res = es.fit_GradientAscent(Xtrain, Xtest, bootstrapping=False, stop='completion', val_frac=0.1)
# res = es.fit_GradientAscentWishart(Xtrain, Xtest, bootstrapping=False, stop='completion', val_frac=0.1)

C = X.T @ X / nsamples # empirical covariance
Cclean = res["Cclean"] # cleaned covariance


In [ ]:
# Load data
paths = read_paths(os.path.join(os.getcwd(), '../paths.txt'))
datadic = getdata(paths[-1])
T, N = np.shape(datadic['032302'])


/Users/ariannaarmanetti/Downloads/data/iPA_183/ts/
652 183


In [4]:
np.shape([el for el in datadic.values()]) 

(136, 652, 183)

In [8]:
resfile = 'all_results_BIO.pickle'

train_fraction = 0.8
results={}

for i,( key, X) in enumerate(datadic.items()):
    if i<3:
        print (i)
        # Compute the corr mat C obtained flattening the training data from all patients
        Cbar = computeCbar([el for el in datadic.values()], train_fraction=train_fraction)
        Xtrain, Xtest = split_train_test(X, train_fraction=train_fraction, standardize=True, seed=i)
        key, value=run.single_run(i, Xtrain, Xtest, Cbar=Cbar)
        results[key]=value

0
1
2


In [7]:
results

{0: {'Shrink_CV_likelihood': {'train_likelihood': 224.4873178312651,
   'test_likelihood': 198.1593034880308,
   'train_completion_error': 0.0038857234261070227,
   'test_completion_error': 0.0053061416676387285,
   'train_pseudolikelihood': 3.6784229880161234,
   'test_pseudolikelihood': 3.5553291800323543,
   'Cclean': array([[ 1.        ,  0.1410098 , -0.14762082, ...,  0.10259685,
            0.10616709,  0.14283884],
          [ 0.1410098 ,  1.        , -0.15279962, ...,  0.32227875,
            0.38575168,  0.5075946 ],
          [-0.14762082, -0.15279962,  1.        , ..., -0.43029132,
           -0.43485895, -0.45010824],
          ...,
          [ 0.10259685,  0.32227875, -0.43029132, ...,  1.        ,
            0.68948242,  0.63954392],
          [ 0.10616709,  0.38575168, -0.43485895, ...,  0.68948242,
            1.        ,  0.86997384],
          [ 0.14283884,  0.5075946 , -0.45010824, ...,  0.63954392,
            0.86997384,  1.        ]]),
   'time': 59.9892604760000

In [6]:
for key, val in results[0].items():
    print (key,val['test_likelihood'])

Shrink_CV_likelihood 198.1593034880308
RIE_CV_likelihood 201.30581685083249
Shrink_CV_completion_error 198.1593034880308
RIE_CV_completion_error 209.72710606993388
